In [1]:
import urllib.request
import json
import pandas as pd
import numpy as np

from keras.layers import Dense , Activation , SimpleRNN , LSTM , Embedding
from keras.models import Sequential , load_model , model_from_json

from datetime import datetime

import re

import os

Using TensorFlow backend.


In [2]:
#■関数

#◆jsonファイルで保存
#引数：保存対象オブジェクト、ファイル名(拡張子なし)
def save_as_json(obj , fname):
    json_fname_with_extension = fname + '.json'
    fw = open(json_fname_with_extension , 'w')
    json.dump(obj , fw , indent = 4)
    fw.close()
    files.download(json_fname_with_extension)

#◆jsonファイルを取得
#引数：対象ファイルのパス
def import_json(fpass):
    fr = open(fpass , 'r')
    f = json.load(fr)
    fr.close()
    return f

In [3]:
#パス指定
folder_this_pgm = os.getcwd()
folder_model_weights = os.path.join(folder_this_pgm , r'model')

#モデル
#--モデル構造
model_file_name = r'model_rnn_qa.json'
model_file_path = os.path.join(folder_model_weights , model_file_name)
with open(model_file_path , 'r') as f:
    model_json = f.read()
model = model_from_json(model_json)
    
#--重み
weights_file_name = r'weights_rnn_qa.hdf5'
weights_file_path = os.path.join(folder_model_weights , weights_file_name)
model.load_weights(weights_file_path)

#単語ID
word2id_q_tmp = import_json(os.path.join(folder_model_weights , r'word2id_q' + r'.json'))
word2id_q = {word:int(id_) for word , id_ in word2id_q_tmp.items()}

word2id_a_tmp = import_json(os.path.join(folder_model_weights , r'word2id_a' + r'.json'))
word2id_a = {word:int(id_) for word , id_ in word2id_a_tmp.items()}

id2word_q_tmp = import_json(os.path.join(folder_model_weights , r'id2word_q' + r'.json'))
id2word_q = {int(id_):word for id_ , word in id2word_q_tmp.items()}

id2word_a_tmp = import_json(os.path.join(folder_model_weights , r'id2word_a' + r'.json'))
id2word_a = {int(id_):word for id_ , word in id2word_a_tmp.items()}

del word2id_q_tmp , word2id_a_tmp

#SEQLEN（モデルのインプットシーケンスの長さ。モデルの定義の一つ。）
SEQLEN = 10

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 10, 128)           2802944   
_________________________________________________________________
lstm_13 (LSTM)               (None, 256)               394240    
_________________________________________________________________
dense_13 (Dense)             (None, 6196)              1592372   
_________________________________________________________________
activation_13 (Activation)   (None, 6196)              0         
Total params: 4,789,556
Trainable params: 4,789,556
Non-trainable params: 0
_________________________________________________________________


In [5]:
#■関数

#※単語idが要素
#wordのリスト　→　入力用行列（1 × 入力word数 × len(word2id_q)）
#※各word が word2id_q に存在すること（＝絶対条件：小文字）
def create_X(word_list , word2id_q = word2id_q):
    wid_list = [word2id_q[word] for word in word_list]
    return_X = np.zeros((1 , len(word_list)) , dtype = int)
    for j in range(len(word_list)):
        return_X[0 , j] = wid_list[j]
    return return_X

#文字列　→　単語を要素とするリスト
def to_word_list(sentence):
    return list(re.sub(r' +' , ' ' , sentence.replace('.' , ' . ').replace(',' , ' , ').replace('"' , ' ').strip()).lower().split(' '))

#word2id_q　に　存在しない単語があるかチェック（全てあれば True）
def exist_words(word_list , word2id_q = word2id_q):
    word2id_q_keys = word2id_q.keys()
    for word in set(word_list):
        if word in word2id_q_keys:
            continue
        else:
            return (False , word)
    else:
        return (True , None)


In [16]:
#■メイン
def main(SEQLEN = SEQLEN , answer_num = 5):
    end_key = '0'
    question = input('検索内容を入力してください。(終了する場合は[' + end_key + '])\n')
    if question == end_key:
        return False
    
    word_list = to_word_list(question)
    #print('word_list' , word_list)
    
    #単語がword2idに存在しない場合は入力し直し
    result_exist_words = exist_words(word_list)
    #print('result_exist_words' , result_exist_words)
    #print('not result_exist_words[0]' , not result_exist_words[0] )
    if not result_exist_words[0]:
        not_exist_word = result_exist_words[1]
        print('【処理不可】' , not_exist_word , 'は当アプリで処理できません。')
        return True
    
    print('...')
    
    #1語目～SEQLEN語目、２語目～(SEQLEN + 1)語目　...　それぞれの予測結果の和をAnswerベクトルとする。
    #--入力データリスト
    X_for_Answer_list = []
    if len(word_list) < SEQLEN:
        X_for_Answer_list.append(create_X(['' for i in range(SEQLEN - len(word_list))] + word_list))
    else :
        for i in range(len(word_list) - SEQLEN + 1):
            X_for_Answer_list.append(create_X(word_list[i : i + SEQLEN]))
    
    #--予測結果
    pred = np.zeros((len(word2id_a)) , dtype = float)
    for X_ in X_for_Answer_list:
        #print('X_' , X_)
        pred += model.predict(X_ , verbose = 0)[0]
        #print('pred_tmp' , model.predict(X_ , verbose = 0)[0])
        #print('pred' , pred)
    
    answer_word = id2word_a[np.argmax(pred)]
    enum_pred = list(enumerate(pred))
    enum_pred.sort(key = lambda x : -x[1])
    
    print('回答：' , answer_word)
    print('【詳細】')
    for i in range(answer_num):
        print((i+1) , '位' , id2word_a[enum_pred[i][0]] , enum_pred[i][1] )
    print('-' * 30)
    
    return True

In [21]:
#処理
print('【開始】' , '-' * 30)

continue_flg = True
while(continue_flg):
    continue_flg = main()

print('【終了】' , '-' * 30)

【開始】 ------------------------------
検索内容を入力してください。(終了する場合は[0])
This car company has been in the news for widespread recalls of its Corollas & other models
...
回答： toyota
【詳細】
1 位 toyota 7.57044577599
2 位 cuba 0.00586745222245
3 位 japan 0.00529290443228
4 位 tennis 0.00409081311955
5 位 evening 0.00301503697483
------------------------------
検索内容を入力してください。(終了する場合は[0])
company has been in the news for its Corollas & other models
...
回答： toyota
【詳細】
1 位 toyota 0.182679247111
2 位 boston 0.0411739954725
3 位 japan 0.0347413046402
4 位 chicago 0.0270601487719
5 位 florida 0.0238484102301
------------------------------
検索内容を入力してください。(終了する場合は[0])
0
【終了】 ------------------------------
